In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
!git clone https://github.com/emiliodelgadouy/utils.git

In [ ]:
from matplotlib import pyplot as plt
from utils.PatchSequence import PatchSequence
from utils.Dataset import Dataset

In [ ]:
dataset = Dataset(lateralize=False, reduced=True, n=500)

In [ ]:
# from google.colab import auth
# auth.authenticate_user()
# !sudo pip install --no-cache-dir -U crcmod
# !gsutil cp gs://helen-data/dumps/dump_cache_reduced.npz ./

In [ ]:
dataset.load_dump('dump_cache_reduced.npz')

In [ ]:
ds_training = dataset[dataset["split"] == "training"]
ds_test = dataset[dataset["split"] == "test"]

In [21]:
# hparams = {
#     'IMG_SIZE': (299, 299),
#     'BATCH_SIZE': 32,
#     'EPOCHS': 1,
#     'l1_reg': 1e-4,
#     'l2_reg': 1e-4,
#     'dropout_rate': 0.4,
#     'learning_rate': 1e-4,
#     'BACKBONE': 'InceptionV3'
# }

In [28]:
from comet_ml import Experiment
import optuna
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

def objective(trial):
    # Crear experimento en Comet
    experiment = Experiment(

        auto_output_logging="simple",  # o "native"
        log_code=False
    )
    experiment.set_name(f"trial_{trial.number}")

    # Hiperparámetros Optuna
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.6)
    l2_reg = trial.suggest_float('l2_reg', 1e-6, 1e-3, log=True)

    IMG_SIZE = (299, 299)
    BATCH_SIZE = 32
    EPOCHS = 3

    experiment.log_parameters({
        "learning_rate": learning_rate,
        "dropout_rate": dropout_rate,
        "l2_reg": l2_reg,
        "backbone": "InceptionV3",
        "batch_size": BATCH_SIZE,
        "img_size": IMG_SIZE,
        "epochs": EPOCHS
    })

    # Generadores
    train_generator = PatchSequence(
        ds_training,
        batch_size=BATCH_SIZE,
        patch_size=IMG_SIZE,
        shuffle=True,
        preprocess_fn=preprocess_input
    )
    val_generator = PatchSequence(
        ds_test,
        batch_size=BATCH_SIZE,
        patch_size=IMG_SIZE,
        shuffle=False,
        preprocess_fn=preprocess_input
    )

    # Backbone
    base = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )
    for layer in base.layers:
        layer.trainable = False

    # Clasificador
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(
        512,
        activation='relu',
        kernel_regularizer=l1_l2(l1=1e-4, l2=l2_reg)
    )(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc_roc', curve='ROC')]
    )

    t0 = time.time()
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        steps_per_epoch=len(train_generator),
        validation_steps=len(val_generator),
        verbose=0
    )
    elapsed = time.time() - t0

    # Log tiempo
    experiment.log_metric("training_time_sec", elapsed)

    # Log métricas por epoch
    for i in range(EPOCHS):
        experiment.log_metric("train_accuracy", history.history["accuracy"][i], step=i)
        experiment.log_metric("train_auc_roc", history.history["auc_roc"][i], step=i)
        experiment.log_metric("train_loss", history.history["loss"][i], step=i)

        experiment.log_metric("val_accuracy", history.history["val_accuracy"][i], step=i)
        experiment.log_metric("val_auc_roc", history.history["val_auc_roc"][i], step=i)
        experiment.log_metric("val_loss", history.history["val_loss"][i], step=i)


    # Evaluación final
    score = model.evaluate(val_generator, steps=len(val_generator), verbose=0)
    experiment.log_metric("final_val_loss", score[0])
    experiment.log_metric("final_val_accuracy", score[1])
    experiment.log_metric("final_val_auc_roc", score[2])

    experiment.end()
    return score[2]  # AUC ROC


In [29]:
study = optuna.create_study(direction='maximize', study_name='optuna_inception')
study.optimize(objective, n_trials=3)

print("🔍 Mejor trial:")
print("  AUC ROC:", study.best_value)
print("  Hiperparámetros:", study.best_params)

[I 2025-08-05 15:29:30,444] A new study created in memory with name: optuna_inception
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorflow, sklearn, keras.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/emiliodelgadouy/prueba/d17df659dc884541ad88ec0169e3b27d

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : trial_0
COMET INFO:     url                   : https://www.comet.com/emiliodelgadouy/prueba/d17df659dc884541ad88ec0169e3b27d
COMET INFO:   Metrics [

🔍 Mejor trial:
  AUC ROC: 0.0
  Hiperparámetros: {'learning_rate': 5.707490682219435e-05, 'dropout_rate': 0.3912327479218696, 'l2_reg': 9.157533584931693e-05}
